# The importance of linear phase when filtering
In certain situations, for example digital communications, it is important that all filters that are designed have linear phase.  In this notebook we design a filter that does not attenuate any frequency, but instead alters only the phase of the input in a non-linear fashion.  As a result of this process, the signal no longer has an obvious digital structure.

### Preamble
Start by importing the Python libraries that we will require

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sps

And define a function that will return true if running in a Jupyter Notebook

In [ ]:
def is_jupyter():
    """Return true if running in a Jupyter Notebook"""
    try:
        if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            return True
        else:
            return False
    except: 
        return False

### User specified parameters

The following parameters can be specified.  

Parameter | Meaning
--------- | -------
<code>symbols</code> | Number of data symbols (e.g. 10)
<code>subsamples</code> | Number of time samples to take for each symbol (e.g. 40)


In [ ]:
symbols = 10
subsamples = 40

### Define poles and corresponding zeros within the unit circle
The following defines an all-pass filter, which means all frequencies are subject to the same gain, and none are attenuated.  It achieves this by ensuring that all poles are matched with a corresponding zero in the reciprocal position.  The phase changes that the filter introduces are a result of the angular location of the poles (and zeros)

In [ ]:
# Define a set of poles that will result in the phase change we desire
# In order to make sure that the output is real, all poles must be
# matched with their complex conjugates.  (The exception is poles that
# are already real).
p = np.zeros(5, dtype=complex)
p[0] = 0.8 * np.exp(1j*2*np.pi/8)
p[1] = p[0].conjugate()
p[2] = 0.9 * np.exp(1j*np.pi/8)
p[3] = p[2].conjugate()
p[4] = 0.95

# Generate zeros that correspond to these, remembering to take the complex
# conjugate so that the pole and zero have the same angle on the Argand
# diagram.  The resulting transfer function will be frequency independent
# (i.e. flat).
z = np.zeros(len(p), dtype=complex)
for i in range(len(p)):
    z[i] = 1 / p[i].conjugate()

# Combine all of the poles and zeros into a single tapped delay line
a = [1]
b = [1]
for i in range(len(p)):
    a = np.convolve(a,[1, -p[i]])
    b = np.convolve(b,[1, -z[i]])

### Normalise the filter to give a gain of 1
We want the gain of the filter (and that means all frequencies) to be 1.  This is easily done by scaling the numerator by the inverse of the sum of all of the filter coefficients

In [ ]:
b = b * sum(a) / sum(b)

### Generate a data sequence
To demonstrate the effect on a digital signal, we will generate a set of data samples

In [ ]:
# randint returns random integers from 1 (inclusive) to 3 (exclusive)
# with output size symbols (i.e. 1 or 2).  We subtract 1 to make this
# 0 or 1
data = np.random.randint(1, 3, symbols) - 1

# Expand it in the time domain so that we can observe the transients
# Kronecker product of two arrays
expanded = np.kron(data, np.ones(subsamples))

# Filter the expanded waveform
output = sps.lfilter(b, a, expanded)

### Plot the resulting signals
After filtering we can observe the difference that altering the phase will make.  It is evident that the waveform is severely distorted, and it would be very difficult to work out what the original data sequence was.

In [ ]:
# For plotting, determine the timescale in terms of symbols
t = np.arange(0, (len(output))) / subsamples

# Plot the original data along with the output from the filter
plt.figure(figsize = (12, 6))
# update label and axis fontsize
plt.rcParams.update({'font.size': 16})
# get current axis
ax = plt.gca()
ax.plot(t, expanded, label = 'Original data')
ax.plot(t, output.real, label = 'Altered phase')

plt.xlabel('Symbol time')
plt.ylabel('Amplitude')
plt.legend()

# Save figure in python or ipython system
if not is_jupyter(): plt.savefig('non_linear_phase_example.pdf')

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.